# Installing Anaconda and xgboost

## original post: http://www.kdnuggets.com/2017/03/simple-xgboost-tutorial-iris-dataset.html

In order to work with the data, I need to install various scientific libraries for python. The best way I have found is to use Anaconda. It simply installs all the libs and helps to install new ones. You can download the installer for Windows, but if you want to install it on a Linux server, you can just copy-paste this into the terminal:

After this, use conda to install pip which you will need for installing xgboost. It is important to install it using Anaconda (in Anaconda’s directory), so that pip installs other libs there as well:

Now, a very important step: install xgboost Python Package dependencies beforehand. I install these ones from experience:

I upgrade my python virtual environment to have no trouble with python versions:

And finally I can install xgboost with pip (keep fingers crossed):

This command installs the latest xgboost version, but if you want to use a previous one, just specify it with:

Now test if everything is has gone well – type python in the terminal and try to import xgboost:

In [1]:
import xgboost as xgb

/home/cs/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


If you see no errors – perfect.

# Xgboost Demo with the Iris Dataset
 
Here I will use the Iris dataset to show a simple example of how to use Xgboost.

First you load the dataset from sklearn, where X will be the data, y – the class labels:

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

Then you split the data into train and test sets with 80-20% split (http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html):

In [3]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Next you need to create the Xgboost specific DMatrix (http://xgboost.readthedocs.io/en/latest/python/python_intro.html) data format from the numpy array. Xgboost can work with numpy arrays directly, load data from svmlignt files and other formats. Here is how to work with numpy arrays:

In [4]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

If you want to use svmlight for less memory consumption, first dump (http://scikit-learn.org/stable/modules/generated/sklearn.datasets.dump_svmlight_file.html) the numpy array into svmlight format and then just pass the filename to DMatrix:

In [5]:
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

Now for the Xgboost to work you need to set the parameters:

In [6]:
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations

Different datasets perform better with different parameters. The result can be really low with one set of params and really good with others. You can look at this Kaggle script (https://www.kaggle.com/tanitter/introducing-kaggle-scripts/grid-search-xgboost-with-scikit-learn) how to search for the best ones. Generally try with eta 0.1, 0.2, 0.3, max_depth in range of 2 to 10 and num_round around few hundred.

# Train
 
Finally the training can begin. You just type:

In [7]:
bst = xgb.train(param, dtrain, num_round)

To see how the model looks you can also dump (http://xgboost.readthedocs.io/en/latest/python/python_intro.html#training) it in human readable form:

In [8]:
bst.dump_model('dump.raw.txt')

And it looks something like this (f0, f1, f2 are features):

You can see that each tree is no deeper than 3 levels as set in the params.

Use the model to predict classes (http://xgboost.readthedocs.io/en/latest/python/python_intro.html#prediction) for the test set:

In [9]:
preds = bst.predict(dtest)

But the predictions look something like this:

Here each column represents class number 0, 1, or 2. For each line you need to select that column where the probability is the highest:

In [10]:
import numpy as np
best_preds = np.asarray([np.argmax(line) for line in preds])

Now you get a nice list with predicted classes:

Determine the precision (http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html) of this prediction:

In [11]:
from sklearn.metrics import precision_score

print precision_score(y_test, best_preds, average='macro')
# >> 1.0

1.0


Perfect! Now save (http://scikit-learn.org/stable/modules/model_persistence.html) the model for later use:

In [12]:
from sklearn.externals import joblib

joblib.dump(bst, 'bst_model.pkl', compress=True)
# bst = joblib.load('bst_model.pkl') # load it later

['bst_model.pkl']

Now you have a working model saved for later use, and ready for more prediction.

See the full code on github (https://gist.github.com/IevaZarina/ef63197e089169a9ea9f3109058a9679) or below: